In [1]:
import datetime
import numpy as np
from copy import deepcopy
import heapq

In [2]:
def read(filename):
    start = open(filename,'r').read().replace('\n', '').replace(' ', '')
    goal = open('goal.txt','r').read().replace('\n', '').replace(' ', '')
    start_state = []
    goal_state = []
    for i in range(0,len(start)):
        start_state.append(int(start[i]))
    for i in range(0,len(goal)):
        goal_state.append(int(goal[i]))
    return start_state, goal_state

In [3]:
class State:

    def __init__(self, state, pre_move = 'root'):
        self.state = np.array([state[:3], state[3:6], state[6:]])
        self.pre_move = pre_move
        self.g = 0
        self.FindZero()

    def __eq__(self, other):
        if isinstance(other, State):
            return np.array_equal(self.state, other.state)
        
    def __hash__(self):
        return hash(tuple(map(tuple, self.state)))

    def FindZero(self):
        index = np.where(self.state == 0)
        coor = list(zip(index[0], index[1]))
        return list(coor[0])

    def ValidMove(self, move):
        if move == 'up':
            if self.FindZero()[0] != 0:
                return True
        elif move == 'down':
            if self.FindZero()[0] != 2:
                return True
        elif move == 'left':
            if self.FindZero()[1] != 0:
                return True
        elif move == 'right':
            if self.FindZero()[1] != 2:
                return True
        else:
            print('invalid move')
            return False
    
    def SavedMove(self, move):
        if self.ValidMove(move):
            saved = deepcopy(self)
            row = self.FindZero()[0]
            col = self.FindZero()[1]
            if move == 'up':
                self.state[row][col] = self.state[row-1][col]
                self.state[row-1][col] = 0
                self.pre_move = 'up'
                self.g += 1
            elif move == 'down':
                self.state[row][col] = self.state[row+1][col]
                self.state[row+1][col] = 0
                self.pre_move = 'down'
                self.g += 1
            elif move == 'left':
                self.state[row][col] = self.state[row][col-1]
                self.state[row][col-1] = 0
                self.pre_move = 'left'
                self.g += 1
            elif move == 'right':
                self.state[row][col] = self.state[row][col+1]
                self.state[row][col+1] = 0
                self.pre_move = 'right'
                self.g += 1
            return saved
        else:
            return False
        
    def Expand(self):
        nodes = []
        action_list = ['up','down','left','right']
        loop_move = ['down','up','right','left']
        for action in range(0,4):
            if(self.ValidMove(action_list[action]) and self.pre_move != loop_move[action]):
                temp = deepcopy(self)
                temp.SavedMove(action_list[action])
                nodes.append(temp)
        return nodes

    def NumMisplaced(self, goal):
      if self.state.shape != goal.state.shape:
        print("Dimension Error")
        return False
      else:
        num_rows, num_cols = self.state.shape
        count = 0
        for i in range(0,num_rows):
            for j in range(0,num_cols):
                if self.state[i][j] != goal.state[i][j] and self.state[i][j] != 0:
                    count += 1
        return count
    
    def ManhattanDist(self, goal):
      if self.state.shape != goal.state.shape:
        print("Dimension Error")
        return False
      else:
        manhattan_dist = 0
        num_rows, num_cols = self.state.shape
        for r in range(0, num_rows):
            for c in range(0, num_cols):
                if self.state[r][c] != 0:
                    goal_index = np.where(goal.state == self.state[r][c])
                    goal_coor = list(zip(goal_index[0], goal_index[1]))
                    goal_coor = list(goal_coor[0])
                    manhattan_dist += abs(r - goal_coor[0]) + abs(c - goal_coor[1])
        return manhattan_dist

In [4]:
def path_find(history, state):

    path = [state.pre_move]
    temp = state

    while temp.pre_move != 'root':
      temp = history[temp]
      path.append(temp.pre_move)
      
    return list(reversed(path))[1:]

def Visited(state, state_list):
    for s in state_list:
        if s == state:
            return True
    return False

def optimized_ASearch(start, goal):
  global n_expand
  queue = [start]
  Inqueue = {start:True}
  f_value = {start: start.g + start.NumMisplaced(goal)}
  history = {start:'root'}
  
  while queue:
    index = None
    target_node = None

    for item in queue: #O(n)
      
      if index == None:
        index = f_value[item]
        target_node = item

      elif f_value[item] < index:
        index = f_value[item]
        target_node = item

    if target_node == goal:
      print('Solved')
      return path_find(history, target_node)
    
    queue.remove(target_node) #O(n)
    Inqueue[target_node] = False

    for node in target_node.Expand():
      n_expand += 1
      if node not in f_value:
        f_value[node] = node.g + node.NumMisplaced(goal)
        # f_value[node] = node.ManhattanDist(goal)
        history[node] = target_node

        # if node not in queue: #O(n) Result in ~60min run time
        #     queue.append(node)
        
        if node not in Inqueue: #O(1) Result in <30min rum time
            queue.append(node)
            Inqueue[node] = True
  return False

In [5]:
n_expand = 0
def main():
    start_state, goal_state = read('input.txt')
    start = State(start_state)
    goal = State(goal_state)
    history = optimized_ASearch(start, goal)
    print(history)
    print(n_expand)

a = datetime.datetime.now()
main()
b = datetime.datetime.now()
print(b - a)